# Demo

let's analyze few homes



### step 1: who might be eligible 

Santa Clara County 车库改造 + detached adu

Alameda County - 车库改造 + detached adu

la county - 车库改造最多

San Francisco County - 车库改造居多

### find price per sqft high and total price not high
# lot size / home size; 
total price / price per sqft





In [17]:
# Read JSON data from county files
import json
import pandas as pd
import os

#display width change to 1000
pd.set_option('display.max_colwidth', 20)



In [18]:
# Define file paths
data_dir = "../data/raw/"
county_files = {
    "Santa Clara": "20250822_114114_santa_clara_county_ca_santa_clara_county.json",
    "Alameda": "20250822_114052_alameda_county_ca_alameda_county.json", 
    "Los Angeles": "20250822_113940_los_angeles_county_ca_los_angeles_county.json"
}

# Load data from each county
county_data = {}
for county_name, filename in county_files.items():
    filepath = os.path.join(data_dir, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        county_data[county_name] = data
        print(f"Loaded {county_name} County: {len(data['listings'])} listings")

# Convert listings to DataFrames for easier analysis
county_dfs = {}
for county_name, data in county_data.items():
    # Extract relevant fields from each listing
    listings = []
    for listing in data['listings']:
        listing_data = {
            'address': listing.get('location', {}).get('address', {}).get('line', ''),
            'city': listing.get('location', {}).get('address', {}).get('city', ''),
            'county_name': listing.get('location', {}).get('county', '').get('name', ''),
            'zip': listing.get('location', {}).get('address', {}).get('postal_code', ''),
            'price': listing.get('list_price', 0),
            'price_per_sqft': listing.get('price_per_sqft', 0),
            'sqft': listing.get('description', {}).get('sqft', 0),
            'lot_sqft': listing.get('description', {}).get('lot_sqft', 0),
            'beds': listing.get('description', {}).get('beds', 0),
            'baths': listing.get('description', {}).get('baths', 0),
            'description': listing.get('description', {}).get('text', ''),
            'year_built': listing.get('description', {}).get('year_built', 0),
            'garage': listing.get('description', {}).get('garage', 0),
            'property_type': listing.get('description', {}).get('type', ''),
            'listing_id': listing.get('listing_id', ''),
            'longitude': listing.get('location', {}).get('address', {}).get('coordinate', {}).get('lon', 0),
            'latitude': listing.get('location', {}).get('address', {}).get('coordinate', {}).get('lat', 0),
            'url': listing.get('href', '')
        }
        listings.append(listing_data)
    
    county_dfs[county_name] = pd.DataFrame(listings)
    print(f"{county_name} DataFrame created with shape: {county_dfs[county_name].shape}")

# Display basic stats for each county
for county_name, df in county_dfs.items():
    print(f"\n{county_name} County Statistics:")
    print(f"  Average price: ${df['price'].mean():,.0f}")
    print(f"  Average price/sqft: ${df['price_per_sqft'].mean():,.0f}")
    print(f"  Average sqft: {df['sqft'].mean():,.0f}")
    print(f"  Average lot sqft: {df['lot_sqft'].mean():,.0f}")

#conver to pandas dataframe
county_dfs = pd.concat(county_dfs.values())

#display
county_dfs.head(5)


Loaded Santa Clara County: 1366 listings
Loaded Alameda County: 1841 listings
Loaded Los Angeles County: 4513 listings
Santa Clara DataFrame created with shape: (1366, 18)
Alameda DataFrame created with shape: (1841, 18)
Los Angeles DataFrame created with shape: (4513, 18)

Santa Clara County Statistics:
  Average price: $2,192,131
  Average price/sqft: $0
  Average sqft: 2,104
  Average lot sqft: 28,001

Alameda County Statistics:
  Average price: $1,557,264
  Average price/sqft: $0
  Average sqft: 2,202
  Average lot sqft: 229,482

Los Angeles County Statistics:
  Average price: $1,346,727
  Average price/sqft: $0
  Average sqft: 2,073
  Average lot sqft: 465,014


,address,city,county_name,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url
0,5939 Pala Mesa Dr,San Jose,Santa Clara,95123,1899888,0,2451.0,4715.0,4,3.0,Welcome to 5939 ...,2000.0,0,single_family,2985606552,-121.813919,37.240225,https://www.real...
1,119 Belcrest Dr,Los Gatos,Santa Clara,95032,2699000,0,1748.0,10376.0,4,2.0,Beautifully reno...,1965.0,0,single_family,2985604175,-121.914295,37.230329,https://www.real...
2,2055 Rucker Ave,Gilroy,Santa Clara,95020,1649777,0,1172.0,306662.0,3,2.0,Brand-new home o...,2025.0,0,single_family,2985604058,-121.556420,37.062856,https://www.real...
3,5850 Chesbro Ave,San Jose,Santa Clara,95123,1499900,0,1777.0,6000.0,3,3.0,Welcome to 5850 ...,1964.0,0,single_family,2985597086,-121.843059,37.244173,https://www.real...
4,1641 Rue Avati,San Jose,Santa Clara,95131,1349888,0,1205.0,6247.0,3,2.0,Welcome to 1641 ...,1973.0,0,single_family,2985596768,-121.880692,37.388961,https://www.real...


In [19]:
#calculate price per sqft / total price
county_dfs['price_per_sqft'] = county_dfs['price'] / county_dfs['sqft']
print(county_dfs.shape)

#analyze county_dfs['price_per_sqft'] 
#plot price per sqft
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.hist(county_dfs['price_per_sqft'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Price per Square Foot')
# plt.xlabel('Price per Square Foot')
# plt.ylabel('Frequency')
# plt.show()

#filter price and unit price
county_dfs = county_dfs[(county_dfs['price']<3000000)&(county_dfs['price']>700000)]
print(county_dfs.shape)

county_dfs = county_dfs[(county_dfs['price_per_sqft']>600)&(county_dfs['price_per_sqft']<1200)]
print(county_dfs.shape)
#calculate  lot sqft/home sqft ratio
county_dfs['lot_sqft_ratio'] = county_dfs['lot_sqft'] / county_dfs['sqft']
print(county_dfs.shape)

#remove those who have lot_sqft_ratio > 100
county_dfs = county_dfs[(county_dfs['lot_sqft_ratio']<20)&(county_dfs['lot_sqft_ratio']>3)]
print(county_dfs.shape)
#select lot sqft > 5000 and home sqft < 2500
county_dfs = county_dfs[(county_dfs['lot_sqft']>5000)&(county_dfs['sqft']<2500)]
print(county_dfs.shape)
#select home year built < 2006
county_dfs = county_dfs[county_dfs['year_built']<2010]
print(county_dfs.shape)


#analyze lot_sqft_ratio
#plot lot_sqft_ratio
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.hist(county_dfs['lot_sqft_ratio'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Lot Square Footage Ratio')
# plt.xlabel('Lot Square Footage Ratio')
# plt.ylabel('Frequency')
# plt.show()

county_dfs = county_dfs.reset_index(drop=True)
county_dfs


#remove those who have adu already

(7720, 18)
(7362, 18)
(4410, 18)
(4410, 19)
(3205, 19)
(2589, 19)
(2563, 19)


,address,city,county_name,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url,lot_sqft_ratio
0,5850 Chesbro Ave,San Jose,Santa Clara,95123,1499900,844.063028,1777.0,6000.0,3,3.0,Welcome to 5850 ...,1964.0,0,single_family,2985597086,-121.843059,37.244173,https://www.real...,3.376477
1,1641 Rue Avati,San Jose,Santa Clara,95131,1349888,1120.239004,1205.0,6247.0,3,2.0,Welcome to 1641 ...,1973.0,0,single_family,2985596768,-121.880692,37.388961,https://www.real...,5.184232
2,5991 Sorrel Ave,San Jose,Santa Clara,95123,1600000,819.672131,1952.0,6120.0,5,2.0,Welcome to 5991 ...,1965.0,0,single_family,2985592836,-121.825358,37.239649,https://www.real...,3.135246
3,266 Vista Ave,San Jose,Santa Clara,95127,900000,813.008130,1107.0,7000.0,2,1.0,PLEASE DO NOT DI...,1920.0,0,single_family,2985590778,-121.825939,37.380543,https://www.real...,6.323397
4,2936 Stutz Way,San Jose,Santa Clara,95148,1650000,1057.692308,1560.0,6000.0,4,2.0,Welcome to 2936 ...,1976.0,0,single_family,2985588295,-121.793813,37.321354,https://www.real...,3.846154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,525 Hacienda Dr,Monrovia,Los Angeles,91016,1125000,1057.330827,1064.0,7410.0,2,1.0,Welcome to this ...,1949.0,0,single_family,2984992140,-117.979528,34.145859,https://www.real...,6.964286
2559,6460 W 84th St,Los Angeles,Los Angeles,90045,1595000,1126.412429,1416.0,6105.0,3,2.0,Do not miss this...,1944.0,0,single_family,2984216377,-118.401232,33.962698,https://www.real...,4.311441
2560,4040 Lehman Rd,La Crescenta,Los Angeles,91214,1275000,728.987993,1749.0,8125.0,3,2.0,Great Opportunit...,1956.0,0,single_family,2984221760,-118.264551,34.231923,https://www.real...,4.645512
2561,2435 Old Grand St,Santa Ana,Orange,92705,950000,611.719253,1553.0,8600.0,3,2.0,Located in the h...,1955.0,0,single_family,2984997579,-117.849471,33.770116,https://www.real...,5.537669


In [20]:
import re
# 定义ADU相关的关键词模式（不区分大小写）
adu_pattern = re.compile(r'\b(adu|accessory\s+dwelling|granny\s+unit|in-?law\s+unit|secondary\s+unit)\b', re.IGNORECASE)
#add True or False to county_dfs and new column name is existing_adu (1 if adu_pattern is found in description, 0 otherwise)
#handle null values
county_dfs['description'] = county_dfs['description'].fillna(' ')
county_dfs['existing_adu'] = county_dfs['description'].apply(lambda x: 1 if adu_pattern.search(x) is not None else 0)

#print count of existing_adu 1 and 0
print(county_dfs[county_dfs['existing_adu']==0].shape)
print(county_dfs[county_dfs['existing_adu']==1].shape)

#display county_dfs
county_dfs[county_dfs['existing_adu']==1]


(2002, 20)
(561, 20)


,address,city,county_name,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url,lot_sqft_ratio,existing_adu
7,818 N 13th St,San Jose,Santa Clara,95112,1270000,965.045593,1316.0,5375.0,3,1.0,Charming 3-Bedro...,1910.0,0,single_family,2985583896,-121.889444,37.357846,https://www.real...,4.084347,1
16,771 N 16th St,San Jose,Santa Clara,95112,999900,824.319868,1213.0,6625.0,3,2.0,Exquisitely remo...,1947.0,0,single_family,2985568233,-121.886224,37.357566,https://www.real...,5.461665,1
22,3305 Hickerson Dr,San Jose,Santa Clara,95127,999900,691.015895,1447.0,5225.0,3,3.0,Charming Home wi...,1962.0,0,single_family,2985545067,-121.811701,37.357361,https://www.real...,3.610919,1
27,82 Hollywood Ave,San Jose,Santa Clara,95112,1288000,658.486708,1956.0,5922.0,4,2.0,"Offering $10, 00...",1920.0,0,single_family,2985536266,-121.875057,37.318495,https://www.real...,3.027607,1
39,336 West Ct,San Jose,Santa Clara,95116,988000,1020.661157,968.0,5625.0,4,3.0,"Rare, flexible s...",1927.0,0,single_family,2985510089,-121.868025,37.354426,https://www.real...,5.810950,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,867 New York Dr,Altadena,Los Angeles,91001,1400000,664.767331,2106.0,7314.0,4,3.0,Welcome to 867 N...,1937.0,0,single_family,2984232934,-118.132487,34.178956,https://www.real...,3.472934,1
2554,13345 Burbank Blvd,Van Nuys,Los Angeles,91401,1100000,947.459087,1161.0,6719.0,3,1.0,SOLD Before PROC...,1948.0,0,single_family,2985005789,-118.424131,34.172431,https://www.real...,5.787252,1
2555,4803 Gambier St,Los Angeles,Los Angeles,90032,729000,767.368421,950.0,7362.0,3,2.0,Think owning a h...,1923.0,0,single_family,2985005429,-118.179680,34.081222,https://www.real...,7.749474,1
2557,557 Black Hills Dr,Claremont,Los Angeles,91711,900000,604.838710,1488.0,9875.0,4,2.0,Welcome to a hom...,1962.0,0,single_family,2985004154,-117.705783,34.115812,https://www.real...,6.636425,1


In [21]:
county_dfs_sample = county_dfs.sample(500)
county_dfs_sample = county_dfs_sample.reset_index(drop=True)
county_dfs_sample

,address,city,county_name,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url,lot_sqft_ratio,existing_adu
0,6051 Santa Barba...,Garden Grove,Orange,92845,1250000,856.751199,1459.0,6968.0,4,2.0,Beautiful Pride ...,1960.0,0,single_family,2985203724,-118.027536,33.791230,https://www.real...,4.775874,0
1,4803 Gambier St,Los Angeles,Los Angeles,90032,729000,767.368421,950.0,7362.0,3,2.0,Think owning a h...,1923.0,0,single_family,2985005429,-118.179680,34.081222,https://www.real...,7.749474,1
2,4618 Sharynne Ln,Torrance,Los Angeles,90505,1599000,1128.440367,1417.0,6890.0,3,2.0,Welcome Home to ...,1955.0,0,single_family,2984431823,-118.363396,33.820095,https://www.real...,4.862385,0
3,1100 N Vinedo Ave,Pasadena,Los Angeles,91107,1399800,1004.881551,1393.0,7027.0,3,2.0,Highly desirable...,1940.0,0,single_family,2984982127,-118.094697,34.165244,https://www.real...,5.044508,0
4,6432 Petit Ave,Lake Balboa,Los Angeles,91406,959000,710.370370,1350.0,7178.0,3,2.0,THIS HOME QUALIF...,1953.0,0,single_family,2984633648,-118.496825,34.187742,https://www.real...,5.317037,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,260 Mara Pl,San Ramon,Contra Costa,94583,1799000,953.870626,1886.0,9900.0,4,3.0,Tucked at the en...,1977.0,0,single_family,2985366786,-121.948157,37.750936,https://www.real...,5.249205,0
496,307 Rheem Blvd,Moraga,Contra Costa,94556,1459888,866.402374,1685.0,14375.0,3,3.0,PRICED TO SELL: ...,1957.0,0,single_family,2984225461,-122.132102,37.862862,https://www.real...,8.531157,0
497,1927 Foxswallow Cir,Pleasanton,Alameda,94566,1749000,837.643678,2088.0,7788.0,4,3.0,"Charming, Move-I...",1971.0,0,single_family,2984030815,-121.878392,37.676979,https://www.real...,3.729885,0
498,3622 Kersten Dr,San Jose,Santa Clara,95124,1998950,1160.830430,1722.0,6100.0,4,3.0,Presenting a new...,1959.0,0,single_family,2983242962,-121.913254,37.262472,https://www.real...,3.542393,1


### Step 2： searching 管辖权 / zoning 

具体实施步骤：

  1. 转换 GPKG 文件坐标系

  # 读取 GPKG 文件
  gdf = gpd.read_file(file_path)

  # 从 EPSG:3857 转换为 EPSG:4326
  gdf_wgs84 = gdf.to_crs('EPSG:4326')

  2. 创建空间点对象

  # 从 county_dfs 创建 GeoDataFrame
  geometry = [Point(xy) for xy in zip(county_dfs.longitude, county_dfs.latitude)]
  properties_gdf = gpd.GeoDataFrame(county_dfs,
                                   crs='EPSG:4326',
                                   geometry=geometry)

  3. 执行空间连接查询

  # 空间连接：找出每个房产所在的 zoning 区域
  result = gpd.sjoin(properties_gdf, gdf_wgs84,
                     how='left',
                     predicate='within')


find zoning / 是否属于city建制区域等内容 其他内容，使用county_dfs_sample这个50rows 数据

In [22]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


In [23]:
# Step 1: Load and convert GPKG zoning data
print("Loading zoning data from GPKG file...")
file_path_north = "../data/raw/zoning-data/ca-north/California_Statewide_Zoning_North_4449911999654225179.gpkg"
file_path_south = "../data/raw/zoning-data/ca-south/California_Statewide_Zoning_South_685864265303796864.gpkg"

# Read GPKG file
gdf_zoning_north = gpd.read_file(file_path_north)
gdf_zoning_south = gpd.read_file(file_path_south)

gdf_zoning = pd.concat([gdf_zoning_north, gdf_zoning_south])

print(f"Loaded {len(gdf_zoning_north)} zoning polygons")
print(f"Original CRS: {gdf_zoning_north.crs}")


print(f"Loaded {len(gdf_zoning_south)} zoning polygons")
print(f"Original CRS: {gdf_zoning_south.crs}")


# Convert from EPSG:3857 to EPSG:4326
print("\nConverting from EPSG:3857 to EPSG:4326...")
gdf_zoning_wgs84 = gdf_zoning.to_crs('EPSG:4326')
print(f"Converted CRS: {gdf_zoning_wgs84.crs}")

# Display sample of zoning data
print("\nSample of zoning data columns:")
print(gdf_zoning_wgs84.columns.tolist())
print("\nFirst 3 records:")
gdf_zoning_wgs84[['County', 'Jurisdiction', 'Code', 'Description', 'New_Class']].head(3)

Loading zoning data from GPKG file...
Loaded 264417 zoning polygons
Original CRS: EPSG:3857
Loaded 304324 zoning polygons
Original CRS: EPSG:3857

Converting from EPSG:3857 to EPSG:4326...
Converted CRS: EPSG:4326

Sample of zoning data columns:
['County', 'Jurisdiction', 'Code', 'Description', 'Classkey', 'ucd_number', 'ucd_description', 'Source', 'Date', 'Version', 'New_Class', 'geometry']

First 3 records:


,County,Jurisdiction,Code,Description,New_Class
0,ALA,ALA,A,Agriculture,Agricultural
1,ALA,ALA,A,Agriculture,Agricultural
2,ALA,ALA,A,Agriculture,Agricultural


In [24]:
# Step 2: Create GeoDataFrame from county_dfs_sample
print("Creating GeoDataFrame from sample properties...")
print(f"Sample size: {len(county_dfs_sample)} properties")

# Create Point geometries from longitude and latitude
geometry = [Point(xy) for xy in zip(county_dfs_sample.longitude, county_dfs_sample.latitude)]

# Create GeoDataFrame
properties_gdf = gpd.GeoDataFrame(
    county_dfs_sample, 
    crs='EPSG:4326',  # WGS84 coordinate system
    geometry=geometry
)

print(f"GeoDataFrame created with CRS: {properties_gdf.crs}")
print(f"Total properties with valid coordinates: {len(properties_gdf)}")

# Display sample
print("\nSample properties:")
properties_gdf[['address', 'city', 'longitude', 'latitude', 'geometry']].head()

Creating GeoDataFrame from sample properties...
Sample size: 500 properties
GeoDataFrame created with CRS: EPSG:4326
Total properties with valid coordinates: 500

Sample properties:


,address,city,longitude,latitude,geometry
0,6051 Santa Barba...,Garden Grove,-118.027536,33.791230,POINT (-118.0275...
1,4803 Gambier St,Los Angeles,-118.179680,34.081222,POINT (-118.1796...
2,4618 Sharynne Ln,Torrance,-118.363396,33.820095,POINT (-118.3634...
3,1100 N Vinedo Ave,Pasadena,-118.094697,34.165244,POINT (-118.0947...
4,6432 Petit Ave,Lake Balboa,-118.496825,34.187742,POINT (-118.4968...


In [25]:
# Step 3: Perform spatial join to find zoning information
print("Performing spatial join to match properties with zoning...")
print("This may take a moment...")

# Create spatial index for faster queries
print("Creating spatial index for zoning data...")
gdf_zoning_wgs84.sindex

# Perform spatial join
# 'within' predicate finds which zoning polygon contains each property point
result_gdf = gpd.sjoin(
    properties_gdf, 
    gdf_zoning_wgs84, 
    how='left',  # Keep all properties even if no zoning match
    predicate='within'  # Check if property point is within zoning polygon
)

# Count matches
matched = result_gdf['index_right'].notna().sum()
total = len(result_gdf)
print(f"\nSpatial join complete!")
print(f"Properties with zoning match: {matched}/{total} ({matched/total*100:.1f}%)")
print(f"Properties without zoning: {total - matched}")

# Display columns
print("\nAvailable columns after join:")
print(result_gdf.columns.tolist())

Performing spatial join to match properties with zoning...
This may take a moment...
Creating spatial index for zoning data...

Spatial join complete!
Properties with zoning match: 500/500 (100.0%)
Properties without zoning: 0

Available columns after join:
['address', 'city', 'county_name', 'zip', 'price', 'price_per_sqft', 'sqft', 'lot_sqft', 'beds', 'baths', 'description', 'year_built', 'garage', 'property_type', 'listing_id', 'longitude', 'latitude', 'url', 'lot_sqft_ratio', 'existing_adu', 'geometry', 'index_right', 'County', 'Jurisdiction', 'Code', 'Description', 'Classkey', 'ucd_number', 'ucd_description', 'Source', 'Date', 'Version', 'New_Class']


In [26]:
# #double check the spatial mapping was correct

# print(result_gdf['county_name'].unique())
# print(result_gdf['County'].unique())

# mapping = {
#     'SCL': 'Santa Clara',
#     'LAN': 'Los Angeles',
#     'ALA': 'Alameda',
#     'ORA': 'Orange',
#     'VEN': 'Ventura',
#     'CCO': 'Contra Costa',
#     'SBR': 'San Bernardino',
# }
# #apply County to county_fullname
# result_gdf['county_fullname'] = result_gdf['County'].map(mapping)
# result_gdf[result_gdf['county_fullname']!=result_gdf['county_name']][['county_name','city','County','county_fullname']]


In [27]:
# Step 4: Analyze zoning results
print("="*70)
print("ZONING ANALYSIS RESULTS")
print("="*70)

# Select relevant columns for display
zoning_columns = ['address', 'city', 'price', 'sqft', 'lot_sqft', 
                  'County', 'Jurisdiction', 'Code', 'Description', 'New_Class']

# Filter to show only matched properties
matched_properties = result_gdf[result_gdf['index_right'].notna()][zoning_columns]

print(f"\nTotal matched properties: {len(matched_properties)}")

# Analyze zoning types
if len(matched_properties) > 0:
    print("\n--- Zoning Type Distribution ---")
    zoning_dist = matched_properties['Description'].value_counts()
    print(zoning_dist.head(10))
    
    print("\n--- Zoning Class Distribution ---")
    class_dist = matched_properties['New_Class'].value_counts()
    print(class_dist.head(10))
    
    print("\n--- Jurisdiction Distribution ---")
    jurisdiction_dist = matched_properties['Jurisdiction'].value_counts()
    print(jurisdiction_dist.head(10))
    
    print("\n--- Sample Matched Properties ---")
    display_cols = ['address', 'city', 'Description', 'New_Class', 'Jurisdiction']
    print(matched_properties[display_cols].head(10))
else:
    print("No properties matched with zoning data")


# Step 5: Check jurisdiction status (incorporated vs unincorporated areas)
print("\n" + "="*70)
print("JURISDICTION ANALYSIS (City vs County)")
print("="*70)

# Analyze if properties are in incorporated cities or unincorporated county areas
if len(matched_properties) > 0:
    # Properties with specific city jurisdiction (incorporated)
    city_jurisdiction = matched_properties[
        ~matched_properties['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False) &
        matched_properties['Jurisdiction'].notna()
    ]
    
    # Properties in county jurisdiction (unincorporated)
    county_jurisdiction = matched_properties[
        matched_properties['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False)
    ]
    
    print(f"\nIncorporated (City) areas: {len(city_jurisdiction)} properties")
    print(f"Unincorporated (County) areas: {len(county_jurisdiction)} properties")
    
    if len(city_jurisdiction) > 0:
        print("\n--- Cities with Properties ---")
        print(city_jurisdiction['Jurisdiction'].value_counts().head())
    
    if len(county_jurisdiction) > 0:
        print("\n--- County Jurisdictions ---")
        print(county_jurisdiction['Jurisdiction'].value_counts().head())
    
    # Show sample properties from each category
    if len(city_jurisdiction) > 0:
        print("\n--- Sample City Properties ---")
        print(city_jurisdiction[['address', 'city', 'Jurisdiction', 'Description']].head(3))
    
    if len(county_jurisdiction) > 0:
        print("\n--- Sample County (Unincorporated) Properties ---")
        print(county_jurisdiction[['address', 'city', 'Jurisdiction', 'Description']].head(3))

# Step 6: Create summary table with all zoning information
print("\n" + "="*70)
print("COMPLETE ZONING SUMMARY TABLE")
print("="*70)

# Create a comprehensive summary DataFrame
if 'index_right' in result_gdf.columns:
    summary_df = result_gdf[['address', 'city', 'price', 'sqft', 'lot_sqft', 'lot_sqft_ratio',
                             'existing_adu', 'County', 'Jurisdiction', 'Code', 
                             'Description', 'New_Class']].copy()
    
    # Add zoning status
    summary_df['has_zoning'] = result_gdf['index_right'].notna()
    
    # Add jurisdiction type
    summary_df['jurisdiction_type'] = 'Unknown'
    summary_df.loc[summary_df['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False), 
                   'jurisdiction_type'] = 'Unincorporated'
    summary_df.loc[~summary_df['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False) & 
                   summary_df['Jurisdiction'].notna(), 
                   'jurisdiction_type'] = 'Incorporated'
    
    print(f"\nTotal properties analyzed: {len(summary_df)}")
    print(f"Properties with zoning info: {summary_df['has_zoning'].sum()}")
    print(f"Properties without zoning: {(~summary_df['has_zoning']).sum()}")
    
    print("\n--- Jurisdiction Type Summary ---")
    print(summary_df['jurisdiction_type'].value_counts())
    
    print("\n--- First 10 Properties with Complete Information ---")
    display_summary = summary_df[summary_df['has_zoning']][
        ['address', 'city', 'price', 'lot_sqft', 'existing_adu', 
         'Description', 'jurisdiction_type']
    ].head(10)
    
    # Format price for better display
    display_summary['price'] = display_summary['price'].apply(lambda x: f"${x:,.0f}")
    display_summary['lot_sqft'] = display_summary['lot_sqft'].apply(lambda x: f"{x:,.0f}")
    
    print(display_summary.to_string(index=False))
else:
    print("No zoning information available in the results")


del matched_properties, display_summary, summary_df

ZONING ANALYSIS RESULTS

Total matched properties: 500

--- Zoning Type Distribution ---
Description
RESIDENTIAL                  105
R-1-8                         46
Single Family Residential     32
                              32
Single-Family Residence       23
R-1                           17
R1                            16
R-1-N                         14
Residential                   13
Single-Family Residential     11
Name: count, dtype: int64

--- Zoning Class Distribution ---
New_Class
Suburban residential    350
Urban residential       111
Multi-family             20
Planned area             13
Other                     3
Exurban residential       2
Open Space                1
Name: count, dtype: int64

--- Jurisdiction Distribution ---
Jurisdiction
Los Angeles    106
San Jose        59
LAN             25
Livermore       16
Long Beach      15
Fullerton       14
ALA             14
Fremont         12
Anaheim         11
Pleasanton      10
Name: count, dtype: int64

--- Sample 

In [35]:
#use a row as example
result_gdf[(result_gdf['lot_sqft']>6500)&(result_gdf['price_per_sqft']>1100)]
example_address = result_gdf[(result_gdf['address']=='956 Tamarack Ave')&(result_gdf['city']=='San Jose')]
example_address = example_address.reset_index(drop=True)
example_address




,address,city,county_name,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,...,Jurisdiction,Code,Description,Classkey,ucd_number,ucd_description,Source,Date,Version,New_Class
0,956 Tamarack Ave,San Jose,Santa Clara,95128,1250000,1106.19469,1130.0,7494.0,2,1.0,...,San Jose,R-1-8,R-1-8,San Jose_Zone_R-1-8,NaN,Suburban residen...,City Hub,7/16/2023,1.2,Suburban residen...


ADU 可行性分析系统 - 产品设计文档
一、产品定位
核心理念
"不是全知的AI，而是智能的成本计算器"
从Berkeley报告可以看出，ADU可行性分析的核心不是找到谁建过（这些数据根本搜不到），而是：

理解复杂的多层法规
识别所有成本影响因素
计算真实的投资回报
提供可执行的解决方案

价值主张

为用户节省100+小时的研究时间
避免踩坑，识别隐藏成本
提供类似专业顾问的分析深度
实时更新，永远是最新法规

二、系统架构
五大核心模块
Module 1: Jurisdiction Checker（管辖权确认）

确认地址属于incorporated city还是county
识别具体zoning（如Berkeley的ES-R）
确认ADU是否允许
找到适用的法规层级（州法、市法、特殊条例）

Module 2: Rules Engine（规则引擎）

解析具体的setback、高度、尺寸要求
识别停车、喷淋等特殊要求
计算实际可建面积
评估不同ADU类型的可行性（独立、附属、JADU、车库改造）

Module 3: Property Analyzer（房产分析）

获取地块基础数据（面积、现有建筑）
收集用户关键输入（后院深度、坡度等）
计算各种ADU选项的可行性
生成具体的建议方案

Module 4: Risk & Cost Analyzer（风险成本分析）

识别所有特殊条件（历史区、断层、溪流、火灾区）
计算每个风险的成本影响
提供缓解策略和专家建议
生成风险矩阵（高/中/低）

Module 5: Financial Calculator（财务计算）

汇总所有成本（硬成本+软成本+风险成本）
估算租金收入（基于地区数据）
计算ROI和投资回收期
提供融资方案建议

三、关键创新点
1. 动态搜索 vs 静态数据库
不维护数据库，每次实时搜索最新信息 (未来：维护数据库)

搜索最新的HCD执法函
搜索最新的法规变化（如AB 1033）
搜索实时的费用更新
搜索特殊分区要求

2. 法规冲突识别
GPT-4o智能分析州法vs地方法冲突

识别可能违法的地方要求
提供法律依据和应对策略
建议何时可以威胁投诉HCD

3. 全面的成本透明度
不只说"能不能"，而是说"要多少钱"

基础建设成本
每个特殊条件的额外成本
软成本明细（设计、许可、顾问）
时间成本（延误的机会成本）




COMPLETE ZONING SUMMARY TABLE

Total properties analyzed: 501
Properties with zoning info: 501
Properties without zoning: 0

--- Jurisdiction Type Summary ---
jurisdiction_type
Incorporated    501
Name: count, dtype: int64

--- First 10 Properties with Complete Information ---
            address        city      price lot_sqft  existing_adu                                            Description jurisdiction_type
15227 Graystone Ave     Norwalk   $869,000    5,068             1                              Single-Family Residential      Incorporated
   1303 N Merona St     Anaheim   $898,000    7,039             1                                                   RS-2      Incorporated
  22849 Leadwell St  West Hills   $830,000    7,499             0                                            RESIDENTIAL      Incorporated
   11373 Rampart Dr      Dublin $1,590,000   13,328             0                                                    R-1      Incorporated
     5841 Arroyo Dr Los An

# 针对设计的时候：先确定能不能后院 （后院尺寸 / 坡度 / setback） - 不能的话你必须只能车库改！


In [ ]:
#step one: follow

# 目的：计划建设的ADU主要用于长期出租、短期出租（Airbnb），还是家庭成员居住？请都按照 improvement来算（长租短租不影响！）

找"规则外"的金矿

  人类专家容易陷入"经验陷阱"：
  - "这个区域不行"
  - "这种地形太难"
  - "这个zoning不允许"

  但AI可以发现：
  - 边界案例：虽然主zoning不允许，但有特殊豁免条款
  - 组合机会：单独看不行，但配合其他政策（比如低收入住房补贴）就可行了
  - 时间套利：现在不行，但6个月后新政策生效就行了


# 价格分析：南加州200-300的均价；而北加州则普遍300-400（车库）；500-600 独立adu

南加州法规比较统一 / 几乎每家都做（北加州还在政变）


# 效果图绘制
示意图  / 如果能2d转3d就更牛了

# 附属功能：未来
- 分析Planning Department的申请数据
- 分析public hearing记录

In [ ]:
'''
Leader Agent 的职责

理解意图：用户是想建 detached ADU？改造车库？还是 JADU？

分配任务：让专门的 agent 去查特定信息

综合判断：收集所有信息后，识别冲突和机会

提出方案：不只是说"能不能"，而是"怎么能"

专业 Agent 分工

我觉得需要这些专业 agent：

Zoning Agent - 查这个地址的分区是什么，该分区允许什么

Code Agent - 建筑规范、电气、管道要求

Site Agent - 地块本身条件（大小、地形、现有建筑）

Neighbor Agent - 影响分析（采光、隐私、视线）

Process Agent - 审批流程、时间、费用

Precedent Agent - 附近类似案例、已批准的项目
'''